# Five-Room Dungeon Generator

The Five-Room Dungeon Generator provides GMs with a way to quickly generate a custom five-room dungeon that can be incorporated into their campaign.  Simply provide the notebook with a concept and the notebook will do the rest.

> [!IMPORTANT]
> You will need an [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download/dotnet/8.0) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .NET Interactive.
> 
>
> To run the LLM prompts and semantic functions, make sure you have an
> - [Azure OpenAI Service Key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=rest-api) or
> - [OpenAI API Key](https://platform.openai.com).

## Step 1 - Define Your Concept

Define your five-room dungeon concept.

In [7]:
// Modify these variables according to your five-room dungeon concept.
var system_message = @"
You are a game master for a Dungeons and Dragons campaign.
Please create a five-room dungeon for a party of 5 level 3 adventurers.
The five-room dungeon should be of medium difficulty.

Output options:
* the five-room dungeon outline, 
* descriptive text for each room in the outline, 
* a list of NPCs and monsters that the party will encounter, 
* a rolltable of possible loot that the party can find.
* Please output the generated information into a single consolidated markdown document.
* please do not format results into json, csv, or other structured data formats
";

var user_message = @"Please generate a five-room dungeon concept based on the following details:
* The five-room dungeon will take place in the town of Hammersend.
* The party must deal with a powerful cult that has taken control of the town of Hammersend.
* The cult is comprised of acolytes and clerics who worship ancient sea witches.
* The cult has formed a partnership with one of the player's brothers, who has been corrupted by the cult's influence.
* The party must infiltrate the cult's evil lair, an abandoned lighthouse.
* The cult will summon a tentacle monster to destroy the party and the town if they are not stopped.
";

## Step 2 - Initial Configuration

Configure the OpenAI or Azure OpenAI endpoints that we will be using.


Choose whether you wish to use the OpenAI or Azure OpenAI service.

In [8]:
bool useAzureOpenAI = false;

Execute the following code which will ask a few questions and save the settings to a local
`settings.json` configuration file, under the [config](config) folder. You can
also edit the file manually if you prefer. **Please keep the file safe.**

In [9]:
#!import config/Settings.cs

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

// Uncomment this if you're using OpenAI and need to set the Org Id
await Settings.AskOrg(useAzureOpenAI);

Settings: OK: AI model configured [config/settings.json]
Settings: OK: API key configured [config/settings.json]


## Step 3 - Instantiate Services

Set up the Microsoft Semantic AI Kernel that will use for generative operations.

In [10]:
// Remove previously installed nuget packages
#r "nuget: Microsoft.SemanticKernel, 1.15.1"
#r "nuget: System.Numerics.Tensors"
#r "nuget: SkiaSharp"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

#!import models/Entry.cs
#!import models/Room.cs
#!import models/NPC.cs
#!import models/RollTable.cs
#!import models/LootTable.cs
#!import models/FiveRoomDungeon.cs

using System.ComponentModel;
using System.Numerics.Tensors;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.TextToImage;

#pragma warning disable SKEXP0001, SKEXP0010

var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
{
    builder.AddAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    builder.AddAzureOpenAITextToImage("dall-e-3", azureEndpoint, apiKey);
}
    
else
{
    builder.AddOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey, orgId);
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
    builder.AddOpenAITextToImage(apiKey, orgId);
}

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins");

var kernel = builder.Build();

// Get chat completion service instance
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

// Get AI service instance used to generate images
var dallE = kernel.GetRequiredService<ITextToImageService>();

// Get AI service instance used to extract embedding from a text
var textEmbedding = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

// Get plugin functions
var pluginFunctions = kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "FiveRoomDungeonPlugin"));

OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var history = new ChatHistory();


Installed Packages Microsoft.SemanticKernel, 1.15.1 SkiaSharp, 2.88.8 System.Numerics.Tensors, 8.0.0

## Step 4 - Generate Outline
Generate the five-room dungeon outline, room descriptions and loot table.

In [11]:
// Add the system message to the chat history
history.AddSystemMessage(system_message);

// Add the user message to the chat history
history.AddUserMessage(user_message);

// 1. Get the response from the AI with automatic function calling
var result = await chatCompletionService.GetChatMessageContentAsync(
    history,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content ?? string.Empty);

// Print the results
Console.WriteLine("Assistant > " + result);


Assistant > ### Five-Room Dungeon Concept: The Lair of the Sea Witches

#### 1. Entrance and Guardian:
The abandoned lighthouse looms before you, its once majestic tower now a crumbling ruin against the backdrop of the stormy sea. The crashing waves below echo ominously, adding to the sense of foreboding that hangs heavy in the air.

At the entrance, two acolytes stand guard, their dark robes billowing in the wind as they scan the horizon with watchful eyes. The flickering torchlight casts eerie shadows across their faces, highlighting the fanatic zeal in their expressions. One of them clutches a wicked-looking dagger, while the other holds a staff adorned with sinister symbols.

Their whispers carry on the wind, a chilling incantation that sends shivers down your spine. The acolytes are on high alert, their senses sharp and attuned to any intruders. To enter the lighthouse, you must find a way to outwit them, whether through stealth or steel. The path ahead is fraught with danger, but

## Step 5 - Convert Outline to Structured Data

The generated content will be more useful to us if we can structure it into a data format that can be consumed by other systems.

In [14]:
// Construct arguments
var arguments = new KernelArguments() { ["input_context"] = result.Content };

// Create the output directory
var outputDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "output");
if (!Directory.Exists(outputDirectory))
{
    Directory.CreateDirectory(outputDirectory);
}

// Parse the Five-Room Dungeon markdown content into JSON
var jsonContent5RD = await kernel.InvokeAsync(pluginFunctions["JSONFormatFiveRoomDungeon"], arguments);
var jsonContent5RDString = jsonContent5RD.ToString().Replace("```json", string.Empty).Replace("```", string.Empty).Trim();
File.WriteAllText(Path.Combine(outputDirectory, "five-room-dungeon.json"), jsonContent5RDString);
FiveRoomDungeon fiveRoomDungeon = JsonSerializer.Deserialize<FiveRoomDungeon>(jsonContent5RDString);

// Parse the Loot markdown content into JSON
var jsonContentLoot = await kernel.InvokeAsync(pluginFunctions["JSONFormatLootTable"], arguments);
var jsonContentLootString = jsonContentLoot.ToString().Replace("```json", string.Empty).Replace("```", string.Empty).Trim();
File.WriteAllText(Path.Combine(outputDirectory, "loot-table.json"), jsonContentLootString);
LootTable lootTable = JsonSerializer.Deserialize<LootTable>(jsonContentLootString);

// Parse the NPC markdown content into JSON
var jsonContentNPCs = await kernel.InvokeAsync(pluginFunctions["JSONFormatNPCs"], arguments);
var jsonContentNPCsString = jsonContentNPCs.ToString().Replace("```json", string.Empty).Replace("```", string.Empty).Trim();
File.WriteAllText(Path.Combine(outputDirectory, "npcs.json"), jsonContentNPCsString);
List<NPC> npcs = JsonSerializer.Deserialize<List<NPC>>(jsonContentNPCsString);

## Step 6 - Use the Structured Data

Now that we have structured data, we can programmatically iterate through objects of interest and perform further action.  In this case, we'll be generating token images for each NPC and consolidate the generated content into a single markdown document.

In [15]:
#pragma warning disable SKEXP0001

var prompt = "A photo-realistic portrait of an NPC with the following description: ";

// Create an httpClient
var httpClient = new HttpClient();

// Output the deserialized objects
foreach (var npc in npcs)
{
    // Use DALL-E 3 to generate an image. OpenAI in this case returns a URL (though you can ask to return a base64 image)
    var imageUrl = await dallE.GenerateImageAsync(prompt + npc.appearance, 256, 256);

    // Download the image to a local file
    using (Stream stream = await httpClient.GetStreamAsync(imageUrl))
    using (MemoryStream memStream = new MemoryStream())
    {
        await stream.CopyToAsync(memStream);
        File.WriteAllBytes(Path.Combine(outputDirectory, $"{npc.name}.png"), memStream.ToArray());
    };
}

// Construct the markdown content
var markdownContent = new StringBuilder();
markdownContent.AppendLine($"# {fiveRoomDungeon.title}");
markdownContent.AppendLine();
markdownContent.AppendLine(fiveRoomDungeon.introduction);
markdownContent.AppendLine();
markdownContent.AppendLine("## Plot Twist");
markdownContent.AppendLine(fiveRoomDungeon.plot_twist);
markdownContent.AppendLine();
markdownContent.AppendLine("## Rooms");
foreach (var room in fiveRoomDungeon.rooms)
{
    markdownContent.AppendLine($"### Room {room.room_number}: {room.title}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Objective**: {room.objective}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Design**: {room.design}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Description**: {room.description}");
    markdownContent.AppendLine();
}
markdownContent.AppendLine("## NPCs");
foreach (var npc in npcs)
{
    markdownContent.AppendLine($"### {npc.name}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Appearance**: {npc.appearance}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Personality**: {npc.personality}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"**Role**: {npc.role}");
    markdownContent.AppendLine();
    markdownContent.AppendLine($"![{npc.name}](./{npc.name}.png)");
    markdownContent.AppendLine();
}
markdownContent.AppendLine("## Loot Table");
markdownContent.AppendLine(lootTable.rolltable.description);
markdownContent.AppendLine();
markdownContent.AppendLine("| Roll | Item | Basic Description | Intermediate Description | Advanced Description |");
markdownContent.AppendLine("|------|------|-------------------|-------------------------|----------------------|");
if (lootTable.rolltable.entries != null)
{
    foreach (var entry in lootTable.rolltable.entries)
    {
        markdownContent.AppendLine($"| {entry.roll} | {entry.item} | {entry.description_basic} | {entry.description_intermediate} | {entry.description_advanced} |");
    }
}

// Output the markdown content to a file
File.WriteAllText(Path.Combine(outputDirectory, "five-room-dungeon.md"), markdownContent.ToString());
